#### Imports
<span style="font-family: Arial; font-size: 11pt;">
Der folgende Code enthält die Imports der aktuellen Saison (22/23), die für dieses Data Science-Projekt verwendet werden:

- `requests`: Diese Bibliothek ermöglicht das Senden von HTTP-Anfragen an Webseiten und die Verarbeitung der erhaltenen Daten.

- `BeautifulSoup` aus `bs4`: Diese Bibliothek wird verwendet, um HTML- oder XML-Dokumente zu parsen. Sie ermöglicht das Durchsuchen von Webseiten und das Extrahieren spezifischer Daten.

- `psycopg2`: Diese Bibliothek dient als Schnittstelle zur PostgreSQL-Datenbank. Sie ermöglicht das Herstellen von Verbindungen zur Datenbank und das Ausführen von Abfragen sowie andere Datenbankoperationen.</span>

In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import psycopg2

#### Kontrollvariable
<span style="font-family: Arial; font-size: 11pt;">
In diesem Codeabschnitt haben wir die Kontrollvariable `conn_Ok` mit dem anfänglichen Wert `True` implementiert, mit der anschliessend überprüft werden soll, ob die Verbindung zur DB steht</span>

In [2]:
# Variablen
conn_Ok = True

#### DB-Verbindung
<span style="font-family: Arial; font-size: 11pt;">
Es wird mit einer Try-Except Methode versucht, eine Verbindung zur PostgreSQL-Datenbank herzustellen.
- Falls die Verbindung erfolgreich hergestellt werden kann, wird ein Cursor-Objekt erstellt und die Variable `conn_Ok` bleibt auf `True`.

Falls ein Verbindungsfehler auftritt, wird eine Fehlermeldung ausgegeben und die Variable `conn_Ok` auf `False` gesetzt.
</span>

In [3]:
# Check if connection to DB possible
try:
    connection = psycopg2.connect(
        dbname='PSQL_ADSFS2023Gruppe15',
        user='ADSFS2023Gruppe15',
        password='ADS_FS_2023_G15!?',
        host='localhost',
        port='5432'
    )

    cursor = connection.cursor()

except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
        conn_Ok = False

#### Datenbereinigung
<span style="font-family: Arial; font-size: 11pt;">
Der folgende Code definiert die Funktion get_team_id, die die Team-ID anhand des Teamnamens aus der PostgreSQL-Datenbank abruft.

- Der übergebene `team_name` wird auf spezifische Teamnamen überprüft und entsprechend angepasst, um den erwarteten Namen in der Datenbank zu verwenden.
- Es wird eine SQL-Abfrage ausgeführt, um die Team-ID aus der Datenbank abzurufen.
- Das Ergebnis der Abfrage wird überprüft und entweder die Team-ID zurückgegeben oder eine Fehlermeldung ausgegeben.
</span>

In [4]:
def get_team_id(team_name):

    if team_name == 'Bor. Mönchengladbach':
        team_name = 'Borussia Mönchengladbach'
    elif team_name == '1899 Hoffenheim':
        team_name = 'TSG 1899 Hoffenheim'
    elif team_name == 'SV Werder Bremen':
        team_name = 'Werder Bremen'
   
        
    # Vergleicht Team Namen und sucht nach ID
    cursor.execute("SELECT team_id FROM bundesliga_mannschaften WHERE mannschaft = %s", (team_name,))
    result = cursor.fetchone()[0]

    if result:
        return result
    else:
        print(f'Keine Übereinstimmung für {team_name} gefunden')


#### Passdaten
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 35 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Passquoten von der HTML-Seite extrahiert.
- Die Team-ID wird mit Hilfe der `get_team_id()`-Funktion abgerufen.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Passdaten) in der Datenbank zu speichern.
- Die Änderungen werden anschliessend in der Datenbank gespeichert.
</span>

In [5]:
if conn_Ok:
    # Anzahl Pässe für die spielTage im angegebenen Range
    for matchday in range(1, 35):
        url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-paesse/'

        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Suche nach allen Mannschaftsnamen und Passquoten
            team_elements = soup.find_all('td', class_='team-name team-name-list')
            pass_complete_elements = soup.find_all('td', class_='team_stats-passes_complete team_stats-passes_complete-list')
            pass_failed_elements = soup.find_all('td', class_='team_stats-passes_failed team_stats-passes_failed-list')
            pass_total_elements = soup.find_all('td', class_='team_stats-passes team_stats-passes-list')
            pass_percentage_elements = soup.find_all('td', class_='team_stats-passes_complete_percentage team_stats-passes_complete_percentage-format')

            # Iteriere über die gefundenen Elemente und extrahiere die gewünschten Informationen
            for team_element,pass_complete_element, pass_failed_element,pass_total_element, pass_percentage_element in zip(team_elements,pass_complete_elements, pass_failed_elements, pass_total_elements, pass_percentage_elements):
                team_name = team_element.text.strip()
                pass_complete = pass_complete_element.text.strip()
                pass_failed = pass_failed_element.text.strip()
                pass_total = pass_total_element.text.strip()
                pass_percentage = pass_percentage_element.text.strip()

                # Speichere die Daten in der PostgreSQL-Datenbank
                insert_data_query = '''
                INSERT INTO bundesliga_pass_stats (matchday, team_id, pass_complete, pass_failed, pass_total, pass_percentage) 
                VALUES (%s, %s, %s,%s, %s, %s);
                '''

                team_id = get_team_id(team_name)

                cursor.execute(insert_data_query, (matchday, team_id, pass_complete, pass_failed, pass_total, float(pass_percentage.replace(',', '.'))))

                connection.commit()
        else:
            print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Passdaten')

#### Torschüsse
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 35 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Schüsse aufs Tor von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Torschüsse) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [6]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-torschuesse/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        shots_elements = soup.find_all('td', class_='team_stats-shots team_stats-shots-list')

        for team_element, shots_element in zip(team_elements, shots_elements):
            team_name = team_element.text.strip()
            shots_total = shots_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_shots_stats (matchday, team_id, shots_total)
                VALUES (%s, %s, %s);
            '''
            team_id = get_team_id(team_name)

            cursor.execute(insert_data_query, (matchday, team_id, shots_total))

            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Torschüsse')

#### Ballkontakte
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 35 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Ballberührungen von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Ballkontakte) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [7]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-ballkontakte/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        touch_elements = soup.find_all('td', class_='team_stats-balls_touched team_stats-balls_touched-list')

        for team_element, touch_element in zip(team_elements, touch_elements):
            team_name = team_element.text.strip()
            touches_total = touch_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_touch_stats (matchday, team_id, touches_total)
                VALUES (%s, %s, %s);
            '''
            team_id = get_team_id(team_name)

            cursor.execute(insert_data_query, (matchday, team_id, touches_total))

            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Ballkontakte')

#### Zweikampfdaten
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 35 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Laufleistungen von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Zweikampfdaten) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [8]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-zweikaempfe/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        duels_elements = soup.find_all('td', class_='team_stats-duels team_stats-duels-list')
        duels_won_elements = soup.find_all('td', class_='team_stats-duels_won team_stats-duels_won-list')
        
        for team_element, duels_element, duels_won_element in zip(team_elements, duels_elements, duels_won_elements):
            team_name = team_element.text.strip()
            duels_total = duels_element.text.strip()
            duels_won = duels_won_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_duels_stats (matchday, team_id, duels_total, duels_won)
                VALUES (%s, %s, %s, %s);
                '''
            team_id =get_team_id(team_name)
        
            cursor.execute(insert_data_query, (matchday, team_id, duels_total, duels_won))
                        
            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Zweikämpfe')

#### Laufleistung
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 35 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Schüsse aufs Tor von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Laufleistung) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [9]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-laufleistung/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        distance_elements = soup.find_all('td', class_='team_stats-tracking_distance team_stats-tracking_distance-format')
        try:
            for team_element, distance_element in zip(team_elements, distance_elements):
                team_name = team_element.text.strip()
                distance_total = distance_element.text.strip()

                insert_data_query = '''
                    INSERT INTO bundesliga_distance_stats (matchday, team_id, distance_total)
                    VALUES (%s, %s, %s);
                    '''
                team_id = get_team_id(team_name)
            
                cursor.execute(insert_data_query, (matchday, team_id, float(distance_total.replace(',', '.'))))
                            
                connection.commit()
        except Exception as e:
            connection.rollback
            print(f'Fehler beim Einfügen der Daten für Spieltag {matchday} und Laufleistung:', str(e))

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Laufleistung')

#### Freistösse
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 35 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Schüsse aufs Tor von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Freistösse) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [10]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-freistoesse/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        freekicks_elements = soup.find_all('td', class_='team_stats-freekicks team_stats-freekicks-list')

        for team_element, freekicks_element in zip(team_elements, freekicks_elements):
            team_name = team_element.text.strip()
            freekicks_total = freekicks_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_freekicks (matchday, team_id, freekicks_total)
                VALUES (%s, %s, %s);
                '''
            team_id = get_team_id(team_name)
        
            cursor.execute(insert_data_query, (matchday, team_id, freekicks_total))
                        
            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Freistösse')

#### Eckbälle
<span style="font-family: Arial; font-size: 11pt;">

Der folgende Codeabschnitt durchläuft eine Schleife von 1 bis 35 (Spieltage) und ruft Daten von einer Webseite ab.

- Eine URL wird für jeden Spieltag generiert, um Daten von einer bestimmten Webseite abzurufen.
- Eine HTTP-Anfrage wird an die URL gesendet, um die Webseite abzurufen.
- Wenn die Antwort erfolgreich ist (200), werden relevante Informationen wie Mannschaftsnamen und Schüsse aufs Tor von der HTML-Seite extrahiert.
- Ein SQL-INSERT-Statement wird erstellt, um die Daten (Spieltag, Team-ID und Eckbälle) in der Datenbank zu speichern.
- Die Änderungen werden in der Datenbank gespeichert.
- Bei auftretenden Fehlern wird die Transaktion rückgängig gemacht und eine entsprechende Fehlermeldung ausgegeben.
</span>

In [ ]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-eckbaelle/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        corner_left_elements = soup.find_all('td', class_='team_stats-corner_kicks_left team_stats-corner_kicks_left-list')
        corner_right_elements = soup.find_all('td', class_='team_stats-corner_kicks_right team_stats-corner_kicks_right-list')

        for team_element, corner_left_element, corner_right_element in zip(team_elements, corner_left_elements, corner_right_elements):
            team_name = team_element.text.strip()
            corner_left = corner_left_element.text.strip()
            corner_right = corner_right_element.text.strip()
            corner_total = corner_left + corner_right

            insert_data_query = '''
                INSERT INTO bundesliga_corners (matchday, team_id, corner_left, corner_right, corner_total)
                VALUES (%s, %s, %s, %s, %s);
                '''
            team_id = get_team_id(team_name)
        
            cursor.execute(insert_data_query, (matchday, team_id, corner_left, corner_right, corner_total))
                        
            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Eckbälle')